In [1]:
import os
from gmsPython.nestingTree.nestingTree import *

## 1. ```Tree```

Collect nesting structure information in pandas indices.

```python
class Tree(name, tree = None, db = None, f = None, **ns)
```

Here (default options):
* ```tree = []```: Nesting structure specified as list of tuples. For each tuple ```x``` we have ```x[0]```= sector, ```x[1]``` = parent node and ```x[2]```= child node.
* ```db = {}```: dictionary database of ```gpy``` symbols (from ```pyDatabases``` package).
* ```f = 'CES'```: Define type of function to apply in the specific nest (used later).
* ```ns```: Dictionary with updates to the namespace of the tree. When we combine nesting trees later on, it will be pertinent that symbols in each tree have specific names. If nothing is provided, the standard convention is to use the syntax ```f"{self.name}_{k}``` for symbol ```k```. 

*Note:* The nesting tree includes fixed names for the fundamental sets - ```s```= sector index, ```n``` = goods index, ```nn``` = alias of ```n```. Every other

In [2]:
name = 'A'
tree = [('s1','KL','K'),
        ('s1','KL','L')]
A = Tree(name, tree = tree)

The ```__call__``` method compiles the tree structure and the main symbols that describe the tree:

In [3]:
A()

The call method establishes 8 symbols based on the tree structure and function:
* ```n```: set of goods
* ```s```: set of sectors
* ```map```: Nesting tree as pd.MultiIndex
* ```knot```: "Knots" in the nesting tree.
* ```branch```: "branche""" in the nesting tree.
* ```input, int, output```: Split goods into types inputs, intermediate goods, and outputs.

All symbols are stored and can be accessed in the database:

In [4]:
A.db

{'A_map': <pyDatabases.gpyDB.database.gpy at 0x15bb00d7e90>,
 'A_knot': <pyDatabases.gpyDB.database.gpy at 0x15bb00e12d0>,
 'A_branch': <pyDatabases.gpyDB.database.gpy at 0x15bb00c9810>,
 'n': <pyDatabases.gpyDB.database.gpy at 0x15bb0506d50>,
 's': <pyDatabases.gpyDB.database.gpy at 0x15bb0507510>,
 'A_input': <pyDatabases.gpyDB.database.gpy at 0x15babc57950>,
 'A_output': <pyDatabases.gpyDB.database.gpy at 0x15bad9979d0>,
 'A_int': <pyDatabases.gpyDB.database.gpy at 0x15bb0507f50>}

The namespace in ```self.ns``` is used when accessing and setting values using the ```__setitem__``` and ```__getitem__``` methods. Accessing a symbol automatically attemps to look up the name in the namespace first, then defaults to the key that we pass. For instance, the namespace details that the ```map``` is named ```map_A``` in this instance:

In [5]:
A['map'].name

'A_map'

In [6]:
A.get('map') # the get method accesses the .vals attribute for the gpy symbol

MultiIndex([('s1', 'KL', 'K'),
            ('s1', 'KL', 'L')],
           names=['s', 'n', 'nn'])

## 2. ```TreeFromData```

We can also infer the structure by loading the share parameter $\mu$:

In [7]:
data = os.path.join(os.getcwd(), 'test', 'treedata.xlsx')
A = TreeFromData(data, 'A')
A()
A.db

{'mu': <pyDatabases.gpyDB.database.gpy at 0x15bad641e50>,
 'A_map': <pyDatabases.gpyDB.database.gpy at 0x15baff38f90>,
 'A_knot': <pyDatabases.gpyDB.database.gpy at 0x15baee79a10>,
 'A_branch': <pyDatabases.gpyDB.database.gpy at 0x15bb00bf010>,
 'n': <pyDatabases.gpyDB.database.gpy at 0x15bb050ad90>,
 's': <pyDatabases.gpyDB.database.gpy at 0x15bb050a290>,
 'A_input': <pyDatabases.gpyDB.database.gpy at 0x15bb0506a50>,
 'A_output': <pyDatabases.gpyDB.database.gpy at 0x15bb050a710>,
 'A_int': <pyDatabases.gpyDB.database.gpy at 0x15bb050b3d0>}

## 3. ```AggTree```

A method that combines ```Tree``` instances into an aggregate tree, i.e. it allow us to assemble nesting trees with different settings (e.g. type of functions to apply in the nesting structure).

```python
class AggTree(name="", trees = None, ws = None, **ns)
```

Here:
* ```name = ""```: Name. Will be added to the namespace as default.
* ```trees={}```: dictionary of ```Tree``` instances (key = name of trees, value = ```Tree``` instance).
* ```ws=None```: Used to initialize database.
* ```ns```: Dictionary with updates to the namespace of symbols relevant for the aggregate nesting tree.

The class always initialize with two additional attributes: 
* ```self.prune = ('n','nn','nnn','s','input','output','int')```: Name of symbols not to carry with us from individual ```Tree``` instances to the aggregate tree.
* ```self.db = GpyDB(ws = ws, alias = [('n','nn'), ('n','nnn')], name = self.name)```: Empty database with alias specification and name from the ```AggTree.name```.

Consider for instance the nesting tree that captures a CES production technology of $X,K,L$ into $KL$ (tree name A) - which then again is split into two outputs $X,Y$ in a CET-like manner (tree name B):

In [8]:
A = Tree('A', tree = [('s1','K','KL'), ('s1','L','KL'), ('s1','X_input','KL')], f = 'CES')
B = Tree('B', tree = [('s1','KL','Y'), ('s1','KL','X')], f = 'CET')

*Note: In the input nest, we replace the name $X$ with ```X_input``` to distinguish this from the output $X$. When compiling the aggregate tree, we make sure to reverse this.*

The aggregate nesting tree:

In [9]:
AB = AggTree(name = 'AB', trees = {tree.name: tree for tree in (A,B)})

The ```__call__(namespace = None)``` method compiles the tree and specifies the relevant symbols in the database. The namespace argument is used to temporarily update set elements: We use it here to replace ```X_input``` with ```X```:

In [10]:
AB(namespace = {'X_input': 'X'}) # 

This call creates a number of symbols, some for the aggregate tree, some for the individual ones:

**Aggregate tree symbols:** Are all defined with syntax ```f'{self.name}_{x}``` with x in:
* ```map```: Concatenation of tree maps
* ```map_spinp```: Part of the map that relies on scale-preserving functions and are input type trees.
* ```map_spout```: Part of the map that relies on scale-preserving functions and are output type trees.
* ```knout```: Knots in the aggregate nesting tree that are output-like types.
* ```kninp```: Knots in the aggregate nesting tree that are input-like types.
* ```spout```: Knots in scale-preserving trees that are output-like.
* ```spinp```: Knots in the scale-preserving trees that are input-like.
* ```input```: Nodes that are inputs in the aggregate tree
* ```output```: Nodes that are outputs in the aggregate tree
* ```int```: intermediate notes in aggregate tree.

**Tree specific symbols:** All defined with syntax ```f'{tree.name}_{x}'``` with x in:
* ```knot_o```: Knots in specific tree that are outputs in aggregate tree (only defined for input-like trees)
* ```knot_no```: Knots in specific tree that *not* outputs in aggregate tree (only defined for input-like trees)
* ```branch2o```: Branches where parent node is an output in aggregate tree (only defined for input-like trees)
* ```branch2no```: Branches where parent node is *not* an output in aggregate tree (only defined for input-like trees)
* ```branch_o```: Branch that is an aggregate output (only defined for output-like trees)
* ```branch_no```: Branch that is *not* an aggregate output (only defined for output-like trees)

In [11]:
AB.db.symbols

{'alias_': <pyDatabases.gpyDB.database.gpy at 0x15bb0530150>,
 'alias_set': <pyDatabases.gpyDB.database.gpy at 0x15bb05250d0>,
 'alias_map2': <pyDatabases.gpyDB.database.gpy at 0x15bb0506850>,
 'n': <pyDatabases.gpyDB.database.gpy at 0x15bad629890>,
 's': <pyDatabases.gpyDB.database.gpy at 0x15bb0542950>,
 'AB_map': <pyDatabases.gpyDB.database.gpy at 0x15bb050b5d0>,
 'AB_map_spinp': <pyDatabases.gpyDB.database.gpy at 0x15bb0525510>,
 'AB_map_spout': <pyDatabases.gpyDB.database.gpy at 0x15bb0542ed0>,
 'AB_knout': <pyDatabases.gpyDB.database.gpy at 0x15bb0532390>,
 'AB_kninp': <pyDatabases.gpyDB.database.gpy at 0x15bb0542410>,
 'AB_spout': <pyDatabases.gpyDB.database.gpy at 0x15bb0543b50>,
 'AB_spinp': <pyDatabases.gpyDB.database.gpy at 0x15bb0542190>,
 'AB_input': <pyDatabases.gpyDB.database.gpy at 0x15bb0540810>,
 'AB_output': <pyDatabases.gpyDB.database.gpy at 0x15bb0531190>,
 'AB_int': <pyDatabases.gpyDB.database.gpy at 0x15bb0542f90>,
 'A_map': <pyDatabases.gpyDB.database.gpy at 0x1

As with the individual trees, we have a namespace attached to the aggregate tree (```self.ns```). This is used when navigating symbols and names of the symbols. 

The ```self.n(item, local = None)``` method is used to navigate the namespaces of the aggregate trees and its individual trees: 

In [12]:
AB.n('map') # if Local = None we access the namespace of the aggregate tree

'AB_map'

In [13]:
AB.n('map', local = 'A') # access namespace of tree 'A'

'A_map'

The ```self.get(item, local = None)``` works through the ```self.n``` method; thus, we access the relevant symbols using:

In [14]:
AB.get('map') # map of aggregate tree

MultiIndex([('s1',  'K', 'KL'),
            ('s1', 'KL',  'X'),
            ('s1', 'KL',  'Y'),
            ('s1',  'L', 'KL'),
            ('s1',  'X', 'KL')],
           names=['s', 'n', 'nn'])

In [15]:
AB.get('map',local = 'A') # nesting tree from part 'A'

MultiIndex([('s1', 'K', 'KL'),
            ('s1', 'L', 'KL'),
            ('s1', 'X', 'KL')],
           names=['s', 'n', 'nn'])

## 4. ```AggTreeFromData```

As with the individual tree, we can also construct this from data. The individual tree names are read from the individual sheets. If no options are provided, we assume all sheets should be used to establish individual trees:

In [16]:
AggTreeData = os.path.join(os.getcwd(), 'test', 'Treedata_.xlsx')
AB = AggTreeFromData(AggTreeData, name = 'AB')(namespace = {'X_input': 'X'})

## 5. ```trimNestingStructure```

*To do: Add trimming of output and mixed tries.*

### 5.1 Input trees:

The option ```keepIntKnots = True``` (not default) keeps intermediate nodes in a nesting tree even though they only have one "child". This can be useful with very complicated nests if e.g. elasticities are defined over intermediate knots.

* A: Tree with (i) Y maps to (KL, B), (ii) KL maps to (K, L).
    * Sparsity 1: No demand for B. Expected result: Y --> (K,L). 
    * Sparsity 2: No demand for L. Expected result: Y --> (B,K) if ```keepIntKnots = False``` else Y --> (KL, B), KL --> (K).
* B: Tree with (i) Y maps to (KL, BM). (ii) KL maps to (K, L). (iii) BM maps to (B,M).
    * Sparsity 1: No demand for B or M. Expected result: Y --> (K,L).

In [17]:
testTrees = {'A': [('s1','Y','KL'), ('s1','Y','B'), ('s1','KL','K'), ('s1','KL','L')],
             'B': [('s1','Y','KL'), ('s1','Y','BM'), ('s1', 'KL','K'), ('s1','KL','L'), ('s1','BM','B'), ('s1','BM','M')]}
tt = {k: Tree(k, tree = testTrees[k])() for k in testTrees}
spars = {'A': [pd.MultiIndex.from_tuples([('s1','K'), ('s1','L')], names = ['s','n']), 
               pd.MultiIndex.from_tuples([('s1','B'), ('s1','K')], names = ['s','n'])],
         'B': [pd.MultiIndex.from_tuples([('s1','K'), ('s1','L')], names = ['s','n'])]}

Test outcome:

In [33]:
tn, i = 'B', 0
m = tt[tn].get('map')
sparsity = spars[tn][i]
trimNestingStructure(m, sparsity, keepIntKnots = False)

MultiIndex([('s1', 'Y', 'K'),
            ('s1', 'Y', 'L')],
           names=['s', 'n', 'nn'])